## Run inference on batch of videos

### Setup the run

The first step is to setup the GPU we'll use to run the inference. With the GPU, inference can take ~30 minutes per 10 minute video, and can be much slower without it. 

In [1]:
import os

The following command will display GPU usage. We can see which are available.

In [2]:
!nvidia-smi

Thu Feb 17 07:07:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 495.44       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:81:00.0 Off |                  Off |
| 30%   30C    P8    19W / 300W |  47975MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:C1:00.0 Off |                  Off |
| 30%   

#### * Choose a free GPU

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

Now we import the packages we'll need

In [4]:
import deeplabcut
#import behaveml
from glob import glob

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


#### * Update paths

Set these paths to where the DLC project was placed, the videos you want to run inference on are, and the pattern that will be used to find the video files.

In [16]:
path_config_file = '/home/blansdel/projects/brett_dlc/animal_pilot/dlc_trained_models/pilot_study-Brett-2021-09-24/config.yaml'
video_path = '/home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/'
video_pattern = 'one_minute_clips_000.mp4'

Settings -- change if you want

In [26]:
create_video = False        # Whether or not to plot detections
shuffle = 1                 # Which shuffle (model version) to use
overwrite = False           # Whether to overwrite current 
track_method = 'ellipse'    # Which tracker to use (ellipse or box). Ellipse is the default, and generally performs better
n_animals = 1               # Number of animals to track (can be different to what the model was trained on)

Now make the list of videos we'll be processing. 

In [52]:
videos = glob(os.path.join(video_path, video_pattern))

Double check things look good

In [47]:
videos

['/home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4']

### Run the inference

May take a while

In [27]:
deeplabcut.analyze_videos(path_config_file, videos, videotype='.avi', n_tracks = n_animals)
if create_video:
    deeplabcut.create_video_with_all_detections(path_config_file, videos)

Using snapshot-100000 for model /home/blansdel/projects/brett_dlc/animal_pilot/dlc_trained_models/pilot_study-Brett-2021-09-24/dlc-models/iteration-4/pilot_studySep24-trainset95shuffle1


/opt/conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/opt/conda/envs/DEEPLABCUT/lib/python3.8/site-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs


2022-02-17 07:35:29.556432: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46717 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c1:00.0, compute capability: 8.6


Starting to analyze %  /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4
Video already analyzed! /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000DLC_dlcrnetms5_pilot_studySep24shuffle1_100000.h5
Using snapshot-100000 for model /home/blansdel/projects/brett_dlc/animal_pilot/dlc_trained_models/pilot_study-Brett-2021-09-24/dlc-models/iteration-4/pilot_studySep24-trainset95shuffle1
Processing...  /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4
Tracklets already computed /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000DLC_dlcrnetms5_pilot_studySep24shuffle1_100000_el.pickle
Set overwrite = True to overwrite.
The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  /home/blansdel

100%|██████████| 1/1 [00:00<00:00, 15768.06it/s]

The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


Filter predictions to smooth the tracks

In [50]:
deeplabcut.stitch_tracklets(path_config_file, videos, n_tracks = n_animals)
deeplabcut.filterpredictions(path_config_file, videos, windowlength = 11)

Processing...  /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4


100%|██████████| 1/1 [00:00<00:00, 21183.35it/s]

Filtering with median model /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4
Saving filtered csv poses!


Make a video of the output

In [51]:
if not create_video:
    deeplabcut.create_labeled_video(path_config_file, videos, filtered = True)

Starting to process video: /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4
Loading /home/blansdel/projects/brett_dlc/animal_pilot/overnight_videos_sample/one_minute_clips_000.mp4 and data.


/opt/conda/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/utils/make_labeled_video.py:592: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df = df.loc(axis=1)[:, individuals]


Duration of video [s]: 60.0, recorded with 30.0 fps!
Overall # of frames: 1800 with cropped frame dimensions: 1600 1200
Generating frames and creating video.


100%|██████████| 1800/1800 [00:33<00:00, 54.19it/s]


## Notes

Can access help for a function by following the command with ?
e.g.

In [53]:
deeplabcut.analyze_videos?

Signature:
deeplabcut.analyze_videos(
    config,
    videos,
    videotype='avi',
    shuffle=1,
    trainingsetindex=0,
    gputouse=None,
    save_as_csv=False,
    destfolder=None,
    batchsize=None,
    cropping=None,
    TFGPUinference=True,
    dynamic=(False, 0.5, 10),
    modelprefix='',
    robust_nframes=False,
    allow_growth=False,
    use_shelve=False,
    auto_track=True,
    n_tracks=None,
    calibrate=False,
    identity_only=False,
)
Docstring:
   Makes prediction based on a trained network. The index of the trained network is specified by parameters in the config file (in particular the variable 'snapshotindex')

   Output: The labels are stored as MultiIndex Pandas Array, which contains the name of the network, body part name, (x, y) label position 

           in pixels, and the likelihood for each frame per body part. These arrays are stored in an efficient Hierarchical Data Format (HDF) 

           in the same directory, where the video is stored. However, if